In [1]:
"""

190 page 기준 8분 정도
휴일 때 대략 100~200 page
평일 때 대략 400~500 page 

한 페이지에 15 기사

"""

'\n\n190 page 기준 8분 정도\n휴일 때 대략 100~200 page\n평일 때 대략 400~500 page \n\n한 페이지에 15 기사\n\n'

In [2]:
from datetime import date
import pandas as pd
from tqdm import notebook
import requests
from bs4 import BeautifulSoup
import re

start_date = date(2022, 1, 1)
end_date = date(2022, 1, 2)


In [3]:
def create_soup(url):

    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
    return soup

In [187]:
def get_articles(year,month,day):

    
    date_data = pd.DataFrame()
    
    year = str(year)
    month = str(month).zfill(2)
    day = str(day).zfill(2)
    date = year+month+day
    
    
    # 마지막 페이지 번호 추출
    url = f"https://news.daum.net/breakingnews/society?page=999&regDate={date}"
    soup = create_soup(url)
    last_page = soup.find("em",attrs ={"class": "num_page"})
    last_page_num = int(re.sub(r'[^0-9]', '', last_page.text))

    
    # 본문 외 내용 제외를 위한 키워드 리스트
    no_get_list = ["[카카오톡]", "[전화]", "[메일]","[온라인 제보]", "재배포 금지", "※ '당신의 제보가 뉴스가 됩니다'", "무단전재", "@"]
    
    replace_list = ["\xa0", "\u2008"]
    
    for i in notebook.tqdm(range(1,last_page_num+1)):
        url = f"https://news.daum.net/breakingnews/society?page={i}&regDate={date}"
        soup = create_soup(url)

        
    
        # 뉴스 리스트가 포함된 속성 추출
        news_list_part = soup.find('ul', attrs={'class':'list_news2 list_allnews'})
        
        # COLUME : press
        press_list = []
        press = news_list_part.find_all('span', attrs={'class':'info_news'})
        for _span in press:
            press_list.append(_span.text.split()[0])
        
        # 뉴스 링크 리스트 추출
        # COLUME : link
        link_list = []
        news_link_list = news_list_part.find_all('a',attrs={"class":"link_thumb"})
        for news_link in news_link_list:
            link_list.append(news_link["href"])
        
        
        
        # COLUME : articleid
        articleid = []
        for link in link_list:
            articleid.append(link.split("/")[-1])
        
        
        
        # 뉴스 링크 리스트 추출
        # COLUME : link
        news_link_list = news_list_part.find_all('a')

        # 링크마다 요청
        # COLUME : title, content ,journalist
        title_list = []
        journalist_list = []
        content_list = []

        for idx, link in enumerate(link_list):
            url = link
            article_soup = create_soup(url)
            article_soup = article_soup.find("article")
            
            ### title
            title = article_soup.find("h3", attrs={'class':"tit_view"})
            title_list.append(title.text)
            
            ### journalist
            journalist = article_soup.find("span", attrs={'class':"txt_info"})
            journalist_list.append(journalist.text)
            
            ### content
            content = article_soup.find("div", attrs={'class':"article_view"})
            
            # 이미지 제거 
            if content.select_one("p.link_figure") != None:
                content.select_one("p.link_figure").decompose()
                
            
            news_contents = content.find_all("p")
            news_content= ""
            
            # p text 확인 후 
            for _p in news_contents:
                skip_p = False
                
                tmp = _p.text
                
                # 주로 \과 관련된 문자를 " "로 변경
                for replace_word in replace_list:
                    tmp = tmp.replace(replace_word," ")
                    
                # tmp = tmp.replace(journalist_list[idx],"")
                
                # 예외가 포함된 문장을 건너뛰기
                for no in no_get_list :
                    if no in tmp :
                        skip_p = True
                        break;
                        
                if skip_p :
                    continue

                        
                news_content += tmp

                
            content_list.append(news_content)
            
                
                
        data =  pd.DataFrame(zip(articleid, title_list,journalist_list,press_list,content_list, link_list))
            
        date_data = pd.concat([date_data, data])
    
    return date_data

In [188]:
# [카카오톡] YTN을 검색해 채널 추가 [전화] 02-398-8585 [메일] social@ytn.co.kr [온라인 제보] www.ytn.co.kr

In [189]:
# from dateutil.rrule import rrule, DAILY

# df = pd.DataFrame()

# for date in notebook.tqdm(list(rrule(DAILY, dtstart=start_date, until=end_date))):
#     df = pd.concat([df, get_articles(date.year, date.month, date.day)])
# df.reset_index(drop=True, inplace=True)

# df = get_articles(date.year, date.month, date.day)

In [190]:
df = get_articles(2022, 1, 1)

  0%|          | 0/10 [00:00<?, ?it/s]

In [172]:
"""
HTTPSConnectionPool(host='v.daum.net', port=443): Max retries exceeded with url: /v/20220101121016727 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020C9A0B23A0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))
"""


"\nHTTPSConnectionPool(host='v.daum.net', port=443): Max retries exceeded with url: /v/20220101121016727 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020C9A0B23A0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))\n"

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141 entries, 0 to 12
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       141 non-null    object
 1   1       141 non-null    object
 2   2       141 non-null    object
 3   3       141 non-null    object
 4   4       141 non-null    object
 5   5       141 non-null    object
dtypes: object(6)
memory usage: 7.7+ KB
